In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [28]:
def read_and_clean(path):
    
   
    df = pd.read_csv(path)


    # convert pandas df to datetime column from 09:15:00+02:00
    # df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S%z')
    del df['time']

    # drop columns with only NaN values
    df = df.dropna(axis=1, how='all')
    # drop rows with only NaN values
    df = df.dropna(axis=0, how='all')
    # drop rows containing any nan values
    df = df.dropna(axis=0, how='any')

    df.reset_index(inplace=True, drop=True)

    # convert to datetime from 5:34:11.166472
    df['receivetime'] = pd.to_datetime(df['receivetime'], format='%H:%M:%S.%f')

    
    df['time_diff'] = df['receivetime'].diff().dt.total_seconds()
    
    


    # get rows where the receivetime difference to the last row is larger than 30 seconds
    mask = (df['time_diff'] > 30)

    filler_rows = []
    # loop over rows in the mask with iterrows
    for index, row in df[mask].iterrows():
        # <- last 5 seconds
        # <- last index
        # <- gap 
        # <- index

        # insert average of last 5 seconds in the gap, divide gap into chunks
        # each chunk has a random gaus time difference to last row of last 5 seconds

        last_index = index - 1
        # select the rows in the last n seconds using the index
        seconds_mean = 15
        seconds_timediff = 100
        mask_last_n_seconds_for_weighted_mean = (df['receivetime'] > df['receivetime'][last_index] - pd.Timedelta(seconds=seconds_mean)) & (df['receivetime'] <= df['receivetime'][last_index])
        mask_last_n_seconds_for_gauss_timediff = (df['receivetime'] > df['receivetime'][last_index] - pd.Timedelta(seconds=seconds_timediff)) & (df['receivetime'] <= df['receivetime'][last_index])

        # select columns which dont have the name receivetime, time, or time_diff
        columns = [col for col in df.columns if col not in ['receivetime', 'time_diff']]

        # get exponentially weighted moving average for rows
        last_n_seconds_exp_weighted_mean = df[mask_last_n_seconds_for_weighted_mean][columns].ewm(span=df[mask_last_n_seconds_for_weighted_mean].shape[0]).mean()
        last_n_seconds_exp_weighted_mean = last_n_seconds_exp_weighted_mean.iloc[-1]
        last_n_seconds_timediffs = df[mask_last_n_seconds_for_gauss_timediff]['time_diff'].tolist()
        
        

        
        time = df.iloc[last_index]['receivetime']
        end_time = df.iloc[index]['receivetime']
        while True:
            # fit a gaussian distribution over the time differences of the last n seconds
            
            gaussian = 0
            while gaussian <= 0:
                gaussian = np.random.normal(np.mean(last_n_seconds_timediffs), np.std(last_n_seconds_timediffs), 1)[0]
            
            time = time + pd.Timedelta(seconds=gaussian)
            if time>= end_time:
                break
            else:
                row = last_n_seconds_exp_weighted_mean.copy()
                row['receivetime'] = time
                # add row to temp_df
                filler_rows.append(row)

        
    # merge temp_df with df
    del df['time_diff']
    filler_df = pd.DataFrame(filler_rows).reset_index(drop=True)
    # convert all rows which are int64 in df to int64 in filler_df
    for col in df.columns:
        if df[col].dtype == 'int64':
            filler_df[col] = filler_df[col].astype('int64')

    df = pd.merge(df, filler_df, how='outer')
    # sort df by receivetime
    df = df.sort_values(by=['receivetime'])
    # reset index
    df = df.reset_index(drop=True)


    # normalize all dataframe columns containing floats to range -1 1 in a for loop
    # df_raw = df.copy()
    df['receivetime'] = df['receivetime'].dt.hour / 24 + df['receivetime'].dt.minute / (24 * 60) + df['receivetime'].dt.second / (24 * 60 * 60) + df['receivetime'].dt.microsecond / (24 * 60 * 60 * 1000000)
    df['receivetime'] = (df['receivetime'] * 2) -1
    
    

    for col in df.columns:
        if col == 'receivetime':
            pass
        elif 'Offers' in col:
            pass
        else: # Price and Volume
            df[col] = (df[col] - df[col].mean()) / df[col].std()
            # df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min()) *2 -1
    return df

def pick_random_subsequences(df,n, len_x, len_y, predict_features):
    """
    df: dataframe
    n: number of subsequences to pick
    seq_len: length of each subsequence
    """
    # get random start indices
    start_indices = np.random.randint(0, df.shape[0] - (len_x+len_y), n)
    # get random end indices
    end_indices = start_indices + (len_x+len_y)
    # get the subsequence for each start and end index
    sequences = [df.iloc[start:end].values.tolist() for start, end in zip(start_indices, end_indices)]
    sequences = np.array(sequences)
    feature_indices = [df.columns.get_loc(col) for col in predict_features]

    X = sequences[:, :len_x, :]
    Y = sequences[:, len_x:, feature_indices]
    return X, Y


def make_train_dev_set(train_csvs, dev_csvs, num_subsequences_per_csv, len_x, len_y, predict_features:list[str], batch_size):
    assert num_subsequences_per_csv > batch_size, "num_subsequences_per_csv must be greater than batch_size"

    
    dfs_train = [read_and_clean(csv) for csv in train_csvs ] 
    dfs_dev = [read_and_clean(csv) for csv in dev_csvs ]

    def data_generator(dfs):
        
        for _ in range(num_subsequences_per_csv//batch_size+1):
            X, Y = [], []
            for df in dfs:
                Xdf, Ydf = pick_random_subsequences(df, batch_size, len_x, len_y, predict_features)
                X.append(Xdf)
                Y.append(Ydf)
            random_order = np.random.permutation(len(X))
            X = np.concatenate(X)[random_order]
            Y = np.concatenate(Y)[random_order]
            for x, y in zip(X, Y):
                yield x, y



    train_dataset = tf.data.Dataset.from_generator(lambda: data_generator(dfs_train), output_types=(tf.float32, tf.float32))
    dev_dataset = tf.data.Dataset.from_generator(lambda: data_generator(dfs_dev), output_types=(tf.float32, tf.float32))

    return train_dataset, dev_dataset

csvs = [
  
"xetrworker1/SAPSE_20042023DMY.csv",
"xetrworker1/SAPSE_21042023DMY.csv",
"xetrworker1/SAPSE_24042023DMY.csv",
"xetrworker1/SAPSE_25042023DMY.csv",
"xetrworker1/SAPSE_26042023DMY.csv",
"xetrworker1/SAPSE_27042023DMY.csv",
"xetrworker1/SAPSE_28042023DMY.csv",
]

Ty = 10
Tx = 100
predict_features = ['aPrice1','aUnit1','askOffers1','bidPrice1','bidUnit1','bidOffers1']
num_subsequences_per_csv = 1000
# given the last Tx we want to predict the next Ty predict_features

batch_size = 2

train, dev = make_train_dev_set(csvs[:-1], csvs[-1:], num_subsequences_per_csv, Tx, Ty, predict_features, batch_size)

In [29]:
first_batch = next(train.batch(batch_size).take(1).as_numpy_iterator())
x, y = first_batch

2023-05-01 19:26:09.361337: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [30]:
x.shape, y.shape

((2, 100, 61), (2, 10, 6))

In [31]:
x_dim = x.shape[-1]
y_dim = y.shape[-1]

In [32]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [33]:
activation_dim = 32
reshaper_enc = Reshape((1, x_dim))
reshaper_dec = Reshape((1, y_dim))
LSTM_cell_encoder = LSTM(activation_dim, return_state = True)
LSTM_cell_decoder = LSTM(activation_dim, return_state = True)
densor = Dense(y_dim, activation='tanh')

In [34]:
def lstm_model(Tx, Ty, LSTM_cell_encoder, LSTM_cell_decoder, densor, reshaper_enc, reshaper_dec, x_dim):
   
    activation_dim = LSTM_cell_encoder.units

    X = Input(shape=(Tx, x_dim))
    a0 = Input(shape=(activation_dim,), name='a')
    c0 = Input(shape=(activation_dim,), name='c')
    a = a0
    c = c0
    
    outputs = []

    # encoder 
    for t in range(Tx):
        x = X[:,t,:]
        x = reshaper_enc(x)
        a, _, c = LSTM_cell_encoder(x, initial_state=[a, c])
      
      
    last_y_out = densor(a)
    outputs.append(last_y_out)
    last_y_out = reshaper_dec(last_y_out)
   
    
    # Y = Input(shape=(Ty, y_dim))
    
    for t in range(Ty-1):
        a, _, c = LSTM_cell_decoder(last_y_out, initial_state=[a, c])
        last_y_out = densor(a)
        
        
        outputs.append(last_y_out)
        last_y_out = reshaper_dec(last_y_out)
    
   
    outputs = tf.transpose(outputs,[1,0,2])

  
        

    model = Model(inputs=[X, a0, c0], outputs=outputs)
    return model

In [35]:
model = lstm_model(Tx, Ty, LSTM_cell_encoder, LSTM_cell_decoder, densor, reshaper_enc, reshaper_dec, x_dim)

2023-05-01 19:26:10.487400: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-01 19:26:10.492658: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-01 19:26:10.497901: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [36]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 100, 61)]    0           []                               
                                                                                                  
 tf.__operators__.getitem_104 (  (None, 61)          0           ['input_2[0][0]']                
 SlicingOpLambda)                                                                                 
                                                                                                  
 tf.__operators__.getitem_103 (  (None, 61)          0           ['input_2[0][0]']                
 SlicingOpLambda)                                                                                 
                                                                                            

In [37]:
model.output_shape

(None, 10, 6)

In [38]:
x,y = train.batch(batch_size).take(1).as_numpy_iterator().next()

2023-05-01 19:26:49.721583: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [39]:
x.shape, y.shape

((2, 100, 61), (2, 10, 6))

In [40]:
from tensorflow.keras.optimizers.legacy import Adam
opt = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])

/home/amos/programming/FraCast/venv/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [41]:


def generator(dataset):
    for x, y in dataset.batch(batch_size):
        a0 = np.zeros((batch_size, activation_dim))
        c0 = np.zeros((batch_size, activation_dim))
        yield [x, a0, c0], y

model.fit(generator(train), epochs=1, verbose = 1)

2023-05-01 19:26:49.908864: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-01 19:26:50.003533: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-01 19:26:51.555386: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradient

1503/1503 [==============================] - 511s 200ms/step - loss: 101.4538 - mae: 3.4304


In [42]:
# predict with model
x,y = dev.batch(batch_size).take(1).as_numpy_iterator().next()
a0 = np.zeros((batch_size, activation_dim))
c0 = np.zeros((batch_size, activation_dim))
y_pred = model.predict([x, a0, c0])


2023-05-01 19:35:21.168266: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [ ]:
y_pred

array([[[ 0.52140343, -0.01067679, -0.37332526,  0.57814896,
         -0.25462517,  0.00516779],
        [ 0.38575014, -0.19225499, -0.2614249 ,  0.5185771 ,
         -0.10263117, -0.14158368]],

       [[ 0.16082022,  0.36035028, -0.39896023,  0.34602478,
          0.05579669, -0.03474039],
        [ 0.2007706 ,  0.10844513, -0.11592612,  0.23262246,
         -0.24540845, -0.05224735]]], dtype=float32)

In [ ]:
model.evaluate(generator(dev))

2023-05-01 19:24:07.284837: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-01 19:24:07.372421: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-05-01 19:24:07.841821: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradient

251/251 [==============================] - 6s 9ms/step - loss: 34.2576 - mae: 3.5997


[34.257568359375, 3.599736213684082]

In [ ]:
x.shape

(2, 5, 61)

In [ ]:
y.shape

(2, 2, 6)